# Dashboard_shipment_pro.ipynb
KPIs and charts for cleaned data. Run ETL first.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
DATA_DIR = Path('.')
CLEANED_CSV = DATA_DIR / 'cleaned_shipments.csv'
REPORT_PDF = DATA_DIR / 'shipment_report.pdf'
if not CLEANED_CSV.exists():
    raise FileNotFoundError('Run ETL_shipment_pro.ipynb first')
df = pd.read_csv(CLEANED_CSV, parse_dates=['shipped_at','delivered_at'])
print('Loaded', len(df), 'rows')

In [ ]:
total_shipments = len(df)
total_claims = int(df['claims'].sum())
claims_per_100 = (total_claims/total_shipments*100) if total_shipments else 0
delivered_ratio = df['status'].eq('delivered').mean() if total_shipments else 0
avg_transit_days = df['transit_days'].dropna().mean()
kpi = {'total_shipments': int(total_shipments),'total_claims': total_claims,'claims_per_100': round(float(claims_per_100),2),'delivered_ratio': round(float(delivered_ratio),4),'avg_transit_days': None if pd.isna(avg_transit_days) else round(float(avg_transit_days),2)}
kpi

In [ ]:
shipments_by_day = df.groupby(df['shipped_at'].dt.date).size().sort_index()
fig = plt.figure(figsize=(8,4))
plt.plot(shipments_by_day.index, shipments_by_day.values)
plt.title('Shipments per day')
plt.xlabel('Date')
plt.ylabel('Number of shipments')
plt.tight_layout()
fig.savefig('chart1_shipments_per_day.png')
plt.show()

In [ ]:
claims_by_branch = df.groupby('branch')['claims'].sum().sort_values(ascending=False)
fig = plt.figure(figsize=(8,4))
plt.bar(claims_by_branch.index, claims_by_branch.values)
plt.title('Claims by branch')
plt.xlabel('Branch')
plt.ylabel('Number of claims')
plt.tight_layout()
fig.savefig('chart2_claims_by_branch.png')
plt.show()

In [ ]:
status_counts = df['status'].fillna('unknown').value_counts()
fig = plt.figure(figsize=(6,6))
plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%')
plt.title('Status distribution')
fig.savefig('chart3_status_distribution.png')
plt.show()

In [ ]:
avg_transit = df.groupby('branch')['transit_days'].mean().dropna().sort_values(ascending=False)
fig = plt.figure(figsize=(8,4))
plt.bar(avg_transit.index, avg_transit.values)
plt.title('Average transit days by branch')
plt.xlabel('Branch')
plt.ylabel('Avg transit days')
plt.tight_layout()
fig.savefig('chart4_avg_transit_by_branch.png')
plt.show()

In [ ]:
branch_counts = df.groupby('branch').size()
branch_claims = df.groupby('branch')['claims'].sum()
claims_rate_100 = (branch_claims/branch_counts*100).sort_values(ascending=False)
fig = plt.figure(figsize=(8,4))
plt.bar(claims_rate_100.index, claims_rate_100.values)
plt.title('Claims per 100 shipments by branch')
plt.xlabel('Branch')
plt.ylabel('Claims per 100 shipments')
plt.tight_layout()
fig.savefig('chart5_claims_rate_per100.png')
plt.show()

In [ ]:
weights = df['weight_kg'].dropna()
fig = plt.figure(figsize=(8,4))
plt.hist(weights, bins=20)
plt.title('Weight distribution')
plt.xlabel('Weight (kg)')
plt.ylabel('Frequency')
plt.tight_layout()
fig.savefig('chart6_weight_hist.png')
plt.show()

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path
with PdfPages(REPORT_PDF) as pdf:
    fig = plt.figure(figsize=(8.27,11.69))
    plt.axis('off')
    text = ('Shipment Analytics Report\n\n' f'Total shipments: {kpi["total_shipments"]}\n' f'Total claims: {kpi["total_claims"]}\n' f'Claims per 100 shipments: {kpi["claims_per_100"]}\n' f'Delivered ratio: {kpi["delivered_ratio"]}\n' f'Average transit days: {kpi["avg_transit_days"]}\n')
    plt.text(0.1,0.9,text,va='top',fontsize=14)
    pdf.savefig(fig)
    plt.close(fig)
    for fname in ['chart1_shipments_per_day.png','chart2_claims_by_branch.png','chart3_status_distribution.png','chart4_avg_transit_by_branch.png','chart5_claims_rate_per100.png','chart6_weight_hist.png']:
        p = Path(fname)
        if p.exists():
            img = plt.imread(p)
            fig = plt.figure(figsize=(8.27,11.69))
            plt.axis('off')
            plt.imshow(img)
            pdf.savefig(fig)
            plt.close(fig)
print('Saved report to', REPORT_PDF)
